In [1]:
from get_embeddings_from_model import extract_embeddings, get_proteometools_data
from pipeline.umap_reducer import UMAPReducer
from config import config
import pandas as pd
import os
import numpy as np
%load_ext autoreload
%autoreload 2

2025-03-08 08:43:28.383391: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 08:43:30.848748: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-08 08:43:30.848858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-08 08:43:30.851286: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 08:43:30.863516: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 08:43:30.866239: I tensorflow/core/platform/cpu_feature_guard.cc:1


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



/cmnfs/home/students/m.celimli/miniconda3/envs/dev_env_mehmet/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/cmnfs/home/students/m.celimli/miniconda3/envs/dev_env_mehmet/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/cmnfs/home/students/m.celimli/miniconda3/envs/dev_env_mehmet/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates 

In [2]:
config

{'data': {'filename': '/cmnfs/proj/prosit/Transformer/no_aug_no_mod_train.parquet',
  'nrows': None,
  'column_name': 'intensities_raw'},
 'umap': {'n_components': 30,
  'n_neighbors': 5,
  'min_dist': 0.1,
  'metric': 'euclidean',
  'embedding_file': None},
 'hdbscan': {'min_cluster_size': 3,
  'min_samples': None,
  'metric': 'euclidean',
  'cluster_selection_epsilon': 0.1,
  'cluster_selection_method': 'eom',
  'max_cluster_size': 100,
  'min_desired_clusters': 1000000,
  'subclustering_min_cluster_size': 5,
  'subclustering_cluster_selection_epsilon': 0.1},
 'knn': {'k': 3, 'use_sqrt': True},
 'sparse_hdbscan': {'min_cluster_size': 3},
 'leiden': {'resolution': 5.0},
 'sparse_dbscan': {'eps': 0.1, 'min_samples': 3}}

In [3]:
modal_embeddings = np.load("embeddings/train_dedup_embeddings_from_model.npy")

In [4]:
np.shape(modal_embeddings)

(4381253, 30)

In [ ]:
# umap_reducer = UMAPReducer(
#     n_components=20,
#     n_neighbors=config["umap"]["n_neighbors"],
#     min_dist=config["umap"]["min_dist"],
#     metric="euclidean",
#     embedding_file = None
# )

In [7]:
# modal_embeddings_20d = umap_reducer.reduce(modal_embeddings)

/cmnfs/home/students/m.celimli/miniconda3/envs/dev_env_mehmet/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/cmnfs/home/students/m.celimli/miniconda3/envs/dev_env_mehmet/lib/python3.10/site-packages/umap/spectral.py:548: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


In [5]:
umap_reducer = UMAPReducer(
    n_components=2,
    n_neighbors=config["umap"]["n_neighbors"],
    min_dist=config["umap"]["min_dist"],
    metric="euclidean",
    embedding_file = None
)

In [ ]:
modal_embeddings_2d = umap_reducer.reduce(modal_embeddings)

/cmnfs/home/students/m.celimli/miniconda3/envs/dev_env_mehmet/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
np.shape(modal_embeddings_2d)

In [ ]:
np.save("embeddings/train_dedup_embeddings_from_model_2_euclidean.npy", modal_embeddings_2d)


In [12]:
np.save("embeddings/train_dedup_embeddings_from_model_20_euclidean.npy", modal_embeddings_20d)
